In [16]:
import os
import numpy as np
import json
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Path to transforms.json
with open(os.path.join('data', 'RedRocks', 'transforms.json'), 'r') as f:
    data = json.load(f)

# Extract camera poses
poses = np.array([[data['frames'][i]['transform_matrix']] 
                        for i in range(len(data['frames']))])
poses = np.squeeze(poses)

In [15]:
positions = poses[:, :3, 3]

# PCA on positions
mean = np.mean(positions, axis=0)
positions = positions - mean
cov = np.dot(positions.T, positions) / positions.shape[0]
U, S, V = np.linalg.svd(cov)

# X and Y directions
X = U[:, 0]
Y = U[:, 1]
Z = U[:, 2]
print(Z)

[-0.0526439  -0.68587789  0.72580999]


In [31]:
# Plot positions and Z vector
fig = go.Figure(data=[go.Scatter3d(x=positions[:, 0], y=positions[:, 1], z=positions[:, 2], mode='markers')])
fig.add_trace(go.Scatter3d(x=[mean[0], mean[0] + Z[0]], y=[mean[1], mean[1] + Z[1]], z=[mean[2], mean[2] + Z[2]], mode='lines'))
fig.update_layout(height=600, width=1200, scene=dict(aspectmode='data'))
fig.show()

In [40]:
# Rotate poses to align with XYZ
T = np.eye(4)
T[:3, 0] = X
T[:3, 1] = Y
T[:3, 2] = Z
poses_rotated = [T.T @ pose for pose in poses]
poses_rotated = np.array(poses_rotated)

In [42]:
positions_rotated = poses_rotated[:, :3, 3]
fig = go.Figure(data=[go.Scatter3d(x=positions_rotated[:, 0], y=positions_rotated[:, 1], z=positions_rotated[:, 2], mode='markers')])
fig.add_trace(go.Scatter3d(x=positions[:, 0], y=positions[:, 1], z=positions[:, 2], mode='markers'))
fig.update_layout(height=600, width=1200, scene=dict(aspectmode='data'))
fig.show()